In [1]:
import urllib.request
import json
import datetime
import pandas as pd
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [16]:
class NaverAPI:
    def get_request_url(self, url):
        self.url = url
        client_id='RqN3GwycVrMQ6MSNNoa6'
        client_secret='a7R6krZYBK'        
        req=urllib.request.Request(self.url)
        req.add_header("X-Naver-Client-Id", client_id)
        req.add_header("X-Naver-Client-Secret",client_secret)       
        try:
            response=urllib.request.urlopen(req)
            print( '[%s] Url Request Success' % (datetime.datetime.now()) )
            return response.read().decode('utf-8')      
        except Exception as err:
            print(err)
            print( '[%s] Url Request Fail [url : %s]' % (datetime.datetime.now(), self.url) )
            return None



    def getNaverMovieResult(self, movieNm, display):
        
        self.movieNm = movieNm
        self.display = display
        url='https://openapi.naver.com/v1/search/movie.json'
        url+='?query=%s'%(urllib.parse.quote(self.movieNm))
        url+='&display=%s'%(self.display)
        retData = self.get_request_url(url)
        
        if(retData == None):
            return None

        else:
            return json.loads(retData)

In [17]:
class RunClassNaverAPI:

    def getKoficData(self, filename):

        self.filename = filename        

        koficData = pd.read_csv(self.filename)
        koficData.fillna('NaN', inplace=True)       
        koficNmDr = koficData[['영화명', '감독']]
        movieList = self.getMovieList(koficNmDr)

        return movieList
    

    def getMovieList(self, dataframe):

        self.dataframe = dataframe
        
        naver = NaverAPI()
        length = len(self.dataframe)
        movieList = []

        
        for idx in range(9):
            print('%d번째 영화 크롤링'%(idx+1))
            koficNm = self.dataframe.iloc[idx, 0]
            koficDr = self.dataframe.iloc[idx, 1]
#             koficDr = koficDr.split(',') 
            
            display_count=100              
            naverResult=naver.getNaverMovieResult(koficNm, display_count)
            movieList.append([koficNm, koficDr, naverResult])
            time.sleep(0.5)


        return movieList

In [18]:
class GetBookExpt:

    def getOribook_expt(self, filename):

        self.filename = filename
        getMovieList = RunClassNaverAPI()        
        movieList = getMovieList.getKoficData(self.filename)

        
        oribook_expt = []    
        for oneList in movieList:

            koficNm = oneList[0]
            koficDr = oneList[1]
            naverResult = oneList[2]

            if naverResult != None:
                naverDictList = naverResult['items']
                for naverDict in naverDictList:

                    movieNm = naverDict['title'].replace('<b>','').replace('</b>','')
                    movieEnNm = naverDict['subtitle']
                    openYr = naverDict['pubDate']
                    movieDr = naverDict['director'].replace('<b>','').replace('</b>','')
                    movieDr = movieDr.rstrip('|').replace('|',',')
                    naverUrl = naverDict['link']                                             
                    response = urlopen(naverUrl)                                       
                    soup = BeautifulSoup(response, 'html.parser')
                    myLi = soup.find('a', attrs={'title':'원작 도서'})

                    if myLi != None:
                        ori_book = 1

                    else:
                        ori_book = 0

#                     print('원작도서 : %d' % (ori_book))
                    naver_ex_pt = soup.select_one('span#interest_cnt_basic')
                    if naver_ex_pt != None:

                        naver_ex_pt = naver_ex_pt.get_text()
                        naver_ex_pt = re.sub('[가-힣]+','', naver_ex_pt)
                        naver_ex_pt = naver_ex_pt.replace(',', '')
                        naver_ex_pt = int(naver_ex_pt)
#                         print('기대지수 : %s' % (naver_ex_pt))


     
                    else :
                        naver_ex_pt = ''

                    oribook_expt.append([movieNm, movieEnNm, openYr, movieDr, ori_book, naver_ex_pt])
#                   print('-'*50)  
                                               
#             print(koficNm)
#             print(koficDr)
#             print('#'*50)

            
#         print(oribook_expt)

        return oribook_expt

In [19]:
getNaverData = GetBookExpt()

In [20]:
naverData = getNaverData.getOribook_expt('NAdata/네이버 크롤링 관련/KOFIC_data(2010-2019).csv')

1번째 영화 크롤링
[2019-06-21 18:03:41.623740] Url Request Success
2번째 영화 크롤링
[2019-06-21 18:03:42.227640] Url Request Success
3번째 영화 크롤링
[2019-06-21 18:03:42.861824] Url Request Success
4번째 영화 크롤링
[2019-06-21 18:03:43.431255] Url Request Success
5번째 영화 크롤링
[2019-06-21 18:03:43.997242] Url Request Success
6번째 영화 크롤링
[2019-06-21 18:03:44.636845] Url Request Success
7번째 영화 크롤링
[2019-06-21 18:03:45.219146] Url Request Success
8번째 영화 크롤링
[2019-06-21 18:03:45.884414] Url Request Success
9번째 영화 크롤링
[2019-06-21 18:03:46.469574] Url Request Success


In [23]:
myframe = pd.DataFrame(naverData, columns=['영화명','영문명','제작년도','감독명','원작도서유무','네이버기대지수'])

In [41]:
myframe.to_csv('네이버크롤링샘플.csv', index=False, encoding='cp949')